<a href="https://colab.research.google.com/github/gaussssss/machinelearning-tpe/blob/main/Reseau_neurone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>NoteBook de TPE d'apprentissage artificiel UY I Niveau 3 Info 2021/2022</h1>

<h3>Liste des membres du TPE</h3>
<ul>
  <li>MOMENE TIYA Florian <strong>21S2785</strong></li>
  <li>TEIDA NOUTSA Remi Raoul <strong>19M2447</strong></li>
  <li>MBOCK NYENGUE Anne Claude <strong>19M2211</strong></li>
  <li>EKOH FOUAPON NJIKAM Yvan <strong>19M2198</strong></li>
</ul>
<br>
<strong><i>Par Dr. Melatagia</i></strong>

Entrainement des réseaux de neurone en python

Pour la première portion du code nous allons créer un tableau à deux colonne ou la première colonne est le vecteur qui contient les entrées de notre reseaux de neurone et la deuxième conlonne sera un vecteur contenant les différentes étiquettes de notre jeu de donnée d'entrainement.Pour ce faire nous allons utiliser la fonction reshape de la librairie numpy.Enfin à la sortir de notre reseaux de neurone,nous allons évaluer l'érreur qui est la différence entre l'étiquette attendu et l'étiquette prédit.

In [ ]:
import numpy as np
from scipy.special import expit as activation_function
from scipy.stats import truncnorm

def truncated_normal(mean=0, sd=1, low=0, upp=10):
  return truncnorm(
  (low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)
class NeuralNetwork:
  def __init__(self,
    no_of_in_nodes,
    no_of_out_nodes,
    no_of_hidden_nodes,
    learning_rate):
    self.no_of_in_nodes = no_of_in_nodes
    self.no_of_out_nodes = no_of_out_nodes
    self.no_of_hidden_nodes = no_of_hidden_nodes
    self.learning_rate = learning_rate
    self.create_weight_matrices()
  def create_weight_matrices(self):
  #""" A method to initialize the weight matrices of the neural network"""
    rad = 1 / np.sqrt(self.no_of_in_nodes)
    X = truncated_normal(mean=0, sd=1, low=-rad, upp=rad)
    self.weights_in_hidden = X.rvs((self.no_of_hidden_nodes,
    self.no_of_in_nodes))
    rad = 1 / np.sqrt(self.no_of_hidden_nodes)
    X = truncated_normal(mean=0, sd=1, low=-rad, upp=rad)
    self.weights_hidden_out = X.rvs((self.no_of_out_nodes,
    self.no_of_hidden_nodes))
  def train(self, input_vector, target_vector):
    #"""input_vector and target_vector can be tuples, lists or ndarrays"""
    # make sure that the vectors have the right shape
    input_vector = np.array(input_vector)
    input_vector = input_vector.reshape(input_vector.size, 1)
    target_vector = np.array(target_vector).reshape(target_vector.size, 1)
    output_vector_hidden = activation_function(self.weights_in_hidden @ input_vector)
    output_vector_network = activation_function(self.weights_hidden_out @ output_vector_hidden)
    output_error = target_vector - output_vector_network
    tmp = output_error * output_vector_network * (1.0 - output_vector_network)
    self.weights_hidden_out += self.learning_rate * (tmp @ output_vector_hidden.T)
    # calculate hidden errors:
    hidden_errors = self.weights_hidden_out.T @ output_error
    # update the weights:
    tmp = hidden_errors * output_vector_hidden * (1.0 - output_vector_hidden)
    self.weights_in_hidden += self.learning_rate * (tmp @ input_vector.T)
  def run(self, input_vector):
    #"""running the network with an input vector 'input_vector'.'input_vector' can be tuple, list or ndarray"""
    # make sure that input_vector is a column vector:
    input_vector = np.array(input_vector)
    input_vector = input_vector.reshape(input_vector.size, 1)
    input4hidden = activation_function(self.weights_in_hidden @ input_vector)
    output_vector_network = activation_function(self.weights_hidden_out @ input4hidden)
    return output_vector_network
  def evaluate(self, data, labels):
   # """
    #Counts how often the actual result corresponds to the
    #target result.
    #A result is considered to be correct, if the index of
    #the maximal value corresponds to the index with the "1"
    #in the one-hot representation,
    #e.g.
    #res = [0.1, 0.132, 0.875]
    #labels[i] = [0, 0, 1]
    #"""
    corrects, wrongs = 0, 0
    for i in range(len(data)):
      res = self.run(data[i])
      res_max = res.argmax()
      if res_max == labels[i].argmax():
        corrects += 1
      else:
        wrongs += 1
    return corrects, wrongs



Pour l'exercution du contenu de notre classe NeuralNetwork,nous allons créer un jeu de donnée grace à la fonction  make_blobs du module sklearn.datasets qui prend comme paramètre le nombre d'échantillon total a généré, le nombre de catégorie ,ainsi qu'un paramètre qui va initialiser le générateur aléatoire.


In [ ]:
from sklearn.datasets import make_blobs
n_samples = 500
blob_centers = ([2, 6], [6, 2], [7, 7])
n_classes = len(blob_centers)
data, labels = make_blobs(n_samples=n_samples,
centers=blob_centers,
random_state=7)

Visualisation des données générées.

In [ ]:
import matplotlib.pyplot as plt
colours = ('green', 'red', "yellow")
fig, ax = plt.subplots()
for n_class in range(n_classes):
  ax.scatter(data[labels==n_class][:, 0],
  data[labels==n_class][:, 1],
  c=colours[n_class],
  s=40,
  label=str(n_class))


Pour échanger les étiquettes, nous pouvons utiliser la commande suivante:

In [ ]:
import numpy as np
labels = np.arange(n_classes) == labels.reshape(labels.size, 1)
labels = labels.astype(np.float)
labels[:7]


Creation du jeu de donnée d'entrainement et de test grace à la fonction train_test_split du module sklearn.model_selection qui prend comme paramètre le jeu de donnée, les étiquettes, les proportions a reservé pour les données d'entrainements et de test ainsi qu'une valeur pour initialiser le générateur aléatoire.

In [ ]:
from sklearn.model_selection import train_test_split
res = train_test_split(data, labels,
train_size=0.8,
test_size=0.2,
random_state=42)
train_data, test_data, train_labels, test_labels = res
train_labels[:10] #Affichage des 10 premiers étiquettes jeu de donnée d'entrainement.

Le code si-dessous permet de créer un réseaux de neurone avec 2 entrées, 5 neurones sur la couche cachée et 3 neurones dans la couvhe de sorties en utilisant un taux d'apprentissage de 0.3.



In [ ]:
simple_network = NeuralNetwork(no_of_in_nodes=2,
no_of_out_nodes=3,
no_of_hidden_nodes=5,
learning_rate=0.3)


Entrainement du réseaux de neurone avec le jeu de donnée d'entrainement et ces étiquettes.

In [ ]:
for i in range(len(train_data)):
  simple_network.train(train_data[i], train_labels[i])


Evaluation du réseau neuronal avec les données d'entrainement ainsi que leurs étiquettes.

In [ ]:
simple_network.evaluate(train_data, train_labels)

Implémentation d'un réseau de neurone biaisé

In [ ]:
import numpy as np
from scipy.stats import truncnorm
from scipy.special import expit as activation_function
def truncated_normal(mean=0, sd=1, low=0, upp=10):
  return truncnorm(
    (low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)


In [ ]:
class NeuralNetwork:
  def __init__(self,
    no_of_in_nodes,
    no_of_out_nodes,
    no_of_hidden_nodes,
    learning_rate,
    bias=None):
    self.no_of_in_nodes = no_of_in_nodes
    self.no_of_hidden_nodes = no_of_hidden_nodes
    self.no_of_out_nodes = no_of_out_nodes
    self.learning_rate = learning_rate
    self.bias = bias
    self.create_weight_matrices()
  def create_weight_matrices(self):
    #""" A method to initialize the weight matrices of the neur
    #al
    #network with optional bias nodes"""
    bias_node = 1 if self.bias else 0
    rad = 1 / np.sqrt(self.no_of_in_nodes + bias_node)
    X = truncated_normal(mean=0, sd=1, low=-rad, upp=rad)
    self.weights_in_hidden = X.rvs((self.no_of_hidden_nodes,
    self.no_of_in_nodes + bias_node))
    rad = 1 / np.sqrt(self.no_of_hidden_nodes + bias_node)
    X = truncated_normal(mean=0, sd=1, low=-rad, upp=rad)
    self.weights_hidden_out = X.rvs((self.no_of_out_nodes,
    self.no_of_hidden_nodes
    + bias_node))
  def train(self, input_vector, target_vector):
    #""" input_vector and target_vector can be tuple, list or n
    #darray """
    # make sure that the vectors have the right shap
    input_vector = np.array(input_vector)
    input_vector = input_vector.reshape(input_vector.size,
    1)
    if self.bias:
    # adding bias node to the end of the input_vector
      input_vector = np.concatenate( (input_vector, [[self.bias]]) )
    target_vector = np.array(target_vector).reshape(target_vector.size, 1)
    output_vector_hidden = activation_function(self.weights_in_hidden @ input_vector)
    if self.bias:
      output_vector_hidden = np.concatenate( (output_vector_hidden, [[self.bias]]) )
    output_vector_network = activation_function(self.weights_hidden_out @ output_vector_hidden)
    output_error = target_vector - output_vector_network
    # update the weights:
    tmp = output_error * output_vector_network * (1.0 - output_vector_network)
    self.weights_hidden_out += self.learning_rate * (tmp @ output_vector_hidden.T)
    # calculate hidden errors:
    hidden_errors = self.weights_hidden_out.T @ output_error
    # update the weights:
    tmp = hidden_errors * output_vector_hidden * (1.0 - output_vector_hidden)
    if self.bias:
      x = (tmp @input_vector.T)[:-1,:] # last row cut off,
    else:
      x = tmp @ input_vector.T
    self.weights_in_hidden += self.learning_rate * x


  def run(self, input_vector):
      #"""
      #running the network with an input vector 'input_vector'.
      #'input_vector' can be tuple, list or ndarray
    # """
      # make sure that input_vector is a column vector:
    input_vector = np.array(input_vector)
    input_vector = input_vector.reshape(input_vector.size, 1)
    if self.bias:
        # adding bias node to the end of the inpuy_vector
      input_vector = np.concatenate( (input_vector, [[1]]) )
    input4hidden = activation_function(self.weights_in_hidden @ input_vector)
    if self.bias:
      input4hidden = np.concatenate( (input4hidden, [[1]]) )
    output_vector_network = activation_function(self.weights_hidden_out @ input4hidden)
    return output_vector_network
  def evaluate(self, data, labels):
    corrects, wrongs = 0, 0
    for i in range(len(data)):
      res = self.run(data[i])
      res_max = res.argmax()
      if res_max == labels[i].argmax():
        corrects += 1
      else:
        wrongs += 1
    return corrects, wrongs


In [ ]:
simple_network = NeuralNetwork(no_of_in_nodes=2,no_of_out_nodes=3,no_of_hidden_nodes=5,learning_rate=0.1,bias=1)
for i in range(len(train_data)):
  simple_network.train(train_data[i], train_labels[i])
simple_network.evaluate(train_data, train_labels)


Utilisation de la fonction d'activation softmax

Création et exécution de la fonction softmax 

In [ ]:
import numpy as np
def softmax(x):
  #""" applies softmax to an input x"""
  e_x = np.exp(x)
  return e_x / e_x.sum()
x = np.array([1, 0, 3, 5])
y = softmax(x)
y, x / x.sum()


In [ ]:
import numpy as np
def softmax(x):
  #""" applies softmax to an input x"""
  e_x = np.exp(x - np.max(x))
  return e_x / e_x.sum()
softmax(x)


In [ ]:
import numpy as np
def softmax(x):
  e_x = np.exp(x)
  return e_x / e_x.sum()
s = softmax(np.array([0, 4, 5]))
si_sj = - s * s.reshape(3, 1)
print(s)
print(si_sj)
s_der = np.diag(s) + si_sj
s_der

In [ ]:
import numpy as np
from scipy.stats import truncnorm
def truncated_normal(mean=0, sd=1, low=0, upp=10):
  return truncnorm(
  (low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)
@np.vectorize
def sigmoid(x):
  return 1 / (1 + np.e ** -x)
def softmax(x):
  e_x = np.exp(x)
  return e_x / e_x.sum()
class NeuralNetwork:
  def __init__(self,
    no_of_in_nodes,
    no_of_out_nodes,
    no_of_hidden_nodes,
    learning_rate,
    softmax=True):
    self.no_of_in_nodes = no_of_in_nodes
    self.no_of_out_nodes = no_of_out_nodes
    self.no_of_hidden_nodes = no_of_hidden_nodes
    self.learning_rate = learning_rate
    self.softmax = softmax
    self.create_weight_matrices()

  def create_weight_matrices(self):
    #""" A method to initialize the weight matrices of the neural network"""
    rad = 1 / np.sqrt(self.no_of_in_nodes)
    X = truncated_normal(mean=0, sd=1, low=-rad, upp=rad)
    self.weights_in_hidden = X.rvs((self.no_of_hidden_nodes,
    self.no_of_in_nodes))
    rad = 1 / np.sqrt(self.no_of_hidden_nodes)
    X = truncated_normal(mean=0, sd=1, low=-rad, upp=rad)
    self.weights_hidden_out = X.rvs((self.no_of_out_nodes,
    self.no_of_hidden_nodes))

  def train(self, input_vector, target_vector):
    #"""input_vector and target_vector can be tuples, lists or ndarrays"""
    # make sure that the vectors have the right shape
    input_vector = np.array(input_vector)
    input_vector = input_vector.reshape(input_vector.size, 1)
    target_vector = np.array(target_vector).reshape(target_vector.size, 1)
    output_vector_hidden = sigmoid(self.weights_in_hidden @ input_vector)
    if self.softmax:
      output_vector_network = softmax(self.weights_hidden_out @ output_vector_hidden)
    else:
      output_vector_network = sigmoid(self.weights_hidden_out @ output_vector_hidden)
    output_error = target_vector - output_vector_network
    if self.softmax:
      ovn = output_vector_network.reshape(output_vector_network.size,)
      si_sj = - ovn * ovn.reshape(self.no_of_out_nodes, 1)
      s_der = np.diag(ovn) + si_sj
      tmp = s_der @ output_error
      self.weights_hidden_out += self.learning_rate * (tmp
      @ output_vector_hidden.T)
    else:
      tmp = output_error * output_vector_network * (1.0 - output_vector_network)
      self.weights_hidden_out += self.learning_rate * (tmp @ output_vector_hidden.T)
      # calculate hidden errors:
    hidden_errors = self.weights_hidden_out.T @ output_error
    # update the weights:
    tmp = hidden_errors * output_vector_hidden * (1.0 - output_vector_hidden)
    self.weights_in_hidden += self.learning_rate * (tmp @ input_vector.T)
  def run(self, input_vector):
    #"""
    #running the network with an input vector 'input_vector'.
    #'input_vector' can be tuple, list or ndarray
    #"""
    # make sure that input_vector is a column vector:
    input_vector = np.array(input_vector)
    input_vector = input_vector.reshape(input_vector.size, 1)
    input4hidden = sigmoid(self.weights_in_hidden @ input_vector)
    if self.softmax:
      output_vector_network = softmax(self.weights_hidden_out @ input4hidden)
    else:
      output_vector_network = sigmoid(self.weights_hidden_out @ input4hidden)
    return output_vector_network
  def evaluate(self, data, labels):
    corrects, wrongs = 0, 0
    for i in range(len(data)):
      res = self.run(data[i])
      res_max = res.argmax()
      if res_max == labels[i]:
        corrects += 1
      else:
        wrongs += 1
      return corrects, wrongs
from sklearn.datasets import make_blobs
n_samples = 300
samples, labels = make_blobs(n_samples=n_samples,
centers=([2, 6], [6, 2]),
random_state=0)


In [ ]:
import matplotlib.pyplot as plt
colours = ('green', 'red', 'blue', 'magenta', 'yellow', 'cyan')
fig, ax = plt.subplots()
for n_class in range(2):
  ax.scatter(samples[labels==n_class][:, 0], samples[labels==n_class][:, 1],
  c=colours[n_class], s=40, label=str(n_class))
size_of_learn_sample = int(n_samples * 0.8)
learn_data = samples[:size_of_learn_sample]
test_data = samples[-size_of_learn_sample:]

simple_network = NeuralNetwork(no_of_in_nodes=2,
  no_of_out_nodes=2,
  no_of_hidden_nodes=5,
  learning_rate=0.3,
  softmax=True)


In [ ]:
for x in [(1, 4), (2, 6), (3, 3), (6, 2)]:
  y = simple_network.run(x)
  print(x, y)

In [ ]:
labels_one_hot = (np.arange(2) == labels.reshape(labels.size, 1))
labels_one_hot = labels_one_hot.astype(np.float)
for i in range(size_of_learn_sample):
  #print(learn_data[i], labels[i], labels_one_hot[i])
  simple_network.train(learn_data[i],
  labels_one_hot[i])
from collections import Counter
evaluation = Counter()
simple_network.evaluate(learn_data, labels)


Matrice de confusion

In [ ]:
import numpy as np
cm = np.array(
[[5825, 1, 49, 23, 7, 46, 30, 12, 21, 26],
[ 1, 6654, 48, 25, 10, 32, 19, 62, 111, 10],
[ 2, 20, 5561, 69, 13, 10, 2, 45, 18, 2],
[ 6, 26, 99, 5786, 5, 111, 1, 41, 110, 79],
[ 4, 10, 43, 6, 5533, 32, 11, 53, 34, 79],
[ 3, 1, 2, 56, 0, 4954, 23, 0, 12, 5],
[ 31, 4, 42, 22, 45, 103, 5806, 3, 34, 3],
[ 0, 4, 30, 29, 5, 6, 0, 5817, 2, 28],
[ 35, 6, 63, 58, 8, 59, 26, 13, 5394, 24],
[ 16, 16, 21, 57, 216, 68, 0, 219, 115, 5693]])


In [ ]:
def precision(label, confusion_matrix):
  col = confusion_matrix[:, label]
  return confusion_matrix[label, label] / col.sum()


In [ ]:
def recall(label, confusion_matrix):
  row = confusion_matrix[label, :]
  return confusion_matrix[label, label] / row.sum()


In [ ]:
def precision_macro_average(confusion_matrix):
  rows, columns = confusion_matrix.shape
  sum_of_precisions = 0
  for label in range(rows):
    sum_of_precisions += precision(label, confusion_matrix)
  return sum_of_precisions / rows


In [ ]:
def recall_macro_average(confusion_matrix):
  rows, columns = confusion_matrix.shape
  sum_of_recalls = 0
  for label in range(columns):
    sum_of_recalls += recall(label, confusion_matrix)
  return sum_of_recalls / columns

In [ ]:
print("label precision recall")
for label in range(10):
  print(f"{label:5d} {precision(label, cm):9.3f} {recall(label,cm):6.3f}")


In [ ]:
print("precision total:", precision_macro_average(cm))
print("recall total:", recall_macro_average(cm))


Détermination de l'exactitude du modèle

In [ ]:
def accuracy(confusion_matrix):
  diagonal_sum = confusion_matrix.trace()
  sum_of_all_elements = confusion_matrix.sum()
  return diagonal_sum / sum_of_all_elements
accuracy(cm)



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
image_size = 28 # width and length
no_of_different_labels = 10 # i.e. 0, 1, 2, 3, ..., 9
image_pixels = image_size * image_size
data_path = "/content/drive/MyDrive/Colab Notebooks/data1/"
train_data = np.loadtxt(data_path + "mnist_train.csv",delimiter=",")
test_data = np.loadtxt(data_path + "mnist_test.csv",delimiter=",")
test_data[:10]
test_data[test_data==255]
test_data.shape


In [ ]:
fac = 0.99 / 255
train_imgs = np.asfarray(train_data[:, 1:]) * fac + 0.01
test_imgs = np.asfarray(test_data[:, 1:]) * fac + 0.01
train_labels = np.asfarray(train_data[:, :1])
test_labels = np.asfarray(test_data[:, :1])


In [ ]:
import numpy as np

lr = np.arange(10)
for label in range(10):
  one_hot = (lr==label).astype(np.int)
  print("label: ", label, " in one-hot representation: ", one_hot)


In [ ]:
lr = np.arange(no_of_different_labels)
# transform labels into one hot representation
train_labels_one_hot = (lr==train_labels).astype(np.float)
test_labels_one_hot = (lr==test_labels).astype(np.float)
# we don't want zeroes and ones in the labels neither:
train_labels_one_hot[train_labels_one_hot==0] = 0.01
train_labels_one_hot[train_labels_one_hot==1] = 0.99
test_labels_one_hot[test_labels_one_hot==0] = 0.01
test_labels_one_hot[test_labels_one_hot==1] = 0.99


In [ ]:
for i in range(10):
  img = train_imgs[i].reshape((28,28))
  plt.imshow(img, cmap="Greys")
  plt.show()


In [ ]:
import pickle
with open("data/mnist/pickled_mnist.pkl", "bw") as fh:
  data = (train_imgs,
    test_imgs,
    train_labels,
    test_labels,
    train_labels_one_hot,
    test_labels_one_hot)
  pickle.dump(data, fh)

In [ ]:
import pickle
with open("data/mnist/pickled_mnist.pkl", "br") as fh:
  data = pickle.load(fh)
train_imgs = data[0]
test_imgs = data[1]
train_labels = data[2]
test_labels = data[3]
train_labels_one_hot = data[4]
test_labels_one_hot = data[5]
image_size = 28 # width and length
no_of_different_labels = 10 # i.e. 0, 1, 2, 3, ..., 9
image_pixels = image_size * image_size


In [ ]:
import numpy as np
@np.vectorize
def sigmoid(x):
  return 1 / (1 + np.e ** -x)
activation_function = sigmoid
from scipy.stats import truncnorm
def truncated_normal(mean=0, sd=1, low=0, upp=10):
  return truncnorm((low - mean) / sd,
(upp - mean) / sd,
loc=mean,
scale=sd)
class NeuralNetwork:
  def __init__(self,
    no_of_in_nodes,
    no_of_out_nodes,
    no_of_hidden_nodes,
    learning_rate):
    self.no_of_in_nodes = no_of_in_nodes
    self.no_of_out_nodes = no_of_out_nodes
    self.no_of_hidden_nodes = no_of_hidden_nodes
    self.learning_rate = learning_rate
    self.create_weight_matrices()
  def create_weight_matrices(self):
    #"""
    #A method to initialize the weight
    #matrices of the neural network
    #"""
    rad = 1 / np.sqrt(self.no_of_in_nodes)
    X = truncated_normal(mean=0,
    sd=1,
    low=-rad,
    upp=rad)
    self.wih = X.rvs((self.no_of_hidden_nodes,
    self.no_of_in_nodes))
    rad = 1 / np.sqrt(self.no_of_hidden_nodes)
    X = truncated_normal(mean=0, sd=1, low=-rad, upp=rad)
    self.who = X.rvs((self.no_of_out_nodes,
    self.no_of_hidden_nodes))
  def train(self, input_vector, target_vector):
    #"""
    #input_vector and target_vector can
    #be tuple, list or ndarray
    #"""
    input_vector = np.array(input_vector, ndmin=2).T
    target_vector = np.array(target_vector, ndmin=2).T
    output_vector1 = np.dot(self.wih,
    input_vector)
    output_hidden = activation_function(output_vector1)
    output_vector2 = np.dot(self.who,
    output_hidden)
    output_network = activation_function(output_vector2)
    output_errors = target_vector - output_network
    # update the weights:
    tmp = output_errors * output_network \
    * (1.0 - output_network)
    tmp = self.learning_rate * np.dot(tmp,
    output_hidden.T)
    self.who += tmp
    # calculate hidden errors:
    hidden_errors = np.dot(self.who.T,
    output_errors)
    # update the weights:
    tmp = hidden_errors * output_hidden * \
    (1.0 - output_hidden)
    self.wih += self.learning_rate \
    * np.dot(tmp, input_vector.T)
  def run(self, input_vector):
    # input_vector can be tuple, list or ndarray
    input_vector = np.array(input_vector, ndmin=2).T
    output_vector = np.dot(self.wih,
    input_vector)
    output_vector = activation_function(output_vector)
    output_vector = np.dot(self.who,
    output_vector)
    output_vector = activation_function(output_vector)
    return output_vector
  def confusion_matrix(self, data_array, labels):
    cm = np.zeros((10, 10), int)
    for i in range(len(data_array)):
      res = self.run(data_array[i])
      res_max = res.argmax()
      target = labels[i][0]
      cm[res_max, int(target)] += 1
    return cm
  def precision(self, label, confusion_matrix):
    col = confusion_matrix[:, label]
    return confusion_matrix[label, label] / col.sum()
  def recall(self, label, confusion_matrix):
    row = confusion_matrix[label, :]
    return confusion_matrix[label, label] / row.sum()
  def evaluate(self, data, labels):
    corrects, wrongs = 0, 0
    for i in range(len(data)):
      res = self.run(data[i])
      res_max = res.argmax()
      if res_max == labels[i]:
        corrects += 1
      else:
        wrongs += 1
    return corrects, wrongs

In [ ]:
ANN = NeuralNetwork(no_of_in_nodes = image_pixels,
no_of_out_nodes = 10,
no_of_hidden_nodes = 100,
learning_rate = 0.1)
for i in range(len(train_imgs)):
  ANN.train(train_imgs[i], train_labels_one_hot[i])


In [ ]:
for i in range(20):
  res = ANN.run(test_imgs[i])
  print(test_labels[i], np.argmax(res), np.max(res))


In [ ]:
corrects, wrongs = ANN.evaluate(train_imgs, train_labels)
print("accuracy train: ", corrects / ( corrects + wrongs))
corrects, wrongs = ANN.evaluate(test_imgs, test_labels)
print("accuracy: test", corrects / ( corrects + wrongs))
cm = ANN.confusion_matrix(train_imgs, train_labels)
print(cm)
for i in range(10):
  print("digit: ", i, "precision: ", ANN.precision(i, cm), "recall: ", ANN.recall(i, cm))


In [ ]:
epochs = 3
NN = NeuralNetwork(no_of_in_nodes = image_pixels,
no_of_out_nodes = 10,
no_of_hidden_nodes = 100,
learning_rate = 0.1)
for epoch in range(epochs):
  print("epoch: ", epoch)
  for i in range(len(train_imgs)):
    NN.train(train_imgs[i],
    train_labels_one_hot[i])
  corrects, wrongs = NN.evaluate(train_imgs, train_labels)
  print("accuracy train: ", corrects / ( corrects + wrongs))
  corrects, wrongs = NN.evaluate(test_imgs, test_labels)
  print("accuracy: test", corrects / ( corrects + wrongs))


In [ ]:
import numpy as np
@np.vectorize
def sigmoid(x):

  return 1 / (1 + np.e ** -x)
activation_function = sigmoid
from scipy.stats import truncnorm
def truncated_normal(mean=0, sd=1, low=0, upp=10):
  return truncnorm((low - mean) / sd,
  (upp - mean) / sd,
  loc=mean,
  scale=sd)
class NeuralNetwork:
  def __init__(self,
      no_of_in_nodes,
      no_of_out_nodes,
      no_of_hidden_nodes,
      learning_rate):
    self.no_of_in_nodes = no_of_in_nodes
    self.no_of_out_nodes = no_of_out_nodes
    self.no_of_hidden_nodes = no_of_hidden_nodes
    self.learning_rate = learning_rate
    self.create_weight_matrices()
  def create_weight_matrices(self):
    #""" A method to initialize the weight matrices of the neur
    #al network"""
    rad = 1 / np.sqrt(self.no_of_in_nodes)
    X = truncated_normal(mean=0,
    sd=1,
    low=-rad,
    upp=rad)
    self.wih = X.rvs((self.no_of_hidden_nodes,
    self.no_of_in_nodes))
    rad = 1 / np.sqrt(self.no_of_hidden_nodes)
    X = truncated_normal(mean=0,
    sd=1,
    low=-rad,
    upp=rad)
    self.who = X.rvs((self.no_of_out_nodes,
    self.no_of_hidden_nodes))
  def train_single(self, input_vector, target_vector):
    
    #"""
    #input_vector and target_vector can be tuple,
    #list or ndarray
    #"""
    output_vectors = []
    input_vector = np.array(input_vector, ndmin=2).T
    target_vector = np.array(target_vector, ndmin=2).T
    output_vector1 = np.dot(self.wih,
    input_vector)
    output_hidden = activation_function(output_vector1)
    output_vector2 = np.dot(self.who,
    output_hidden)
    output_network = activation_function(output_vector2)
    output_errors = target_vector - output_network
    # update the weights:
    tmp = output_errors * output_network * \
    (1.0 - output_network)
    tmp = self.learning_rate * np.dot(tmp,
    output_hidden.T)
    self.who += tmp
    # calculate hidden errors:
    hidden_errors = np.dot(self.who.T,
    output_errors)
    # update the weights:
    tmp = hidden_errors * output_hidden * (1.0 - output_hidden)
    self.wih += self.learning_rate * np.dot(tmp, input_vector.T)
  def train(self, data_array,
    labels_one_hot_array,
    epochs=1,
    intermediate_results=False):
    intermediate_weights = []
    for epoch in range(epochs):
      print("*", end="")
      for i in range(len(data_array)):
        self.train_single(data_array[i],
        labels_one_hot_array[i])
        if intermediate_results:
          intermediate_weights.append((self.wih.copy(),
          self.who.copy()))
    return intermediate_weights
  def confusion_matrix(self, data_array, labels):
    cm = {}
    for i in range(len(data_array)):
      res = self.run(data_array[i])
      res_max = res.argmax()
      target = labels[i][0]
      if (target, res_max) in cm:
        cm[(target, res_max)] += 1
      else:
        cm[(target, res_max)] = 1
    return cm
  def run(self, input_vector):
    #""" input_vector can be tuple, list or ndarray """
    input_vector = np.array(input_vector, ndmin=2).T
    output_vector = np.dot(self.wih,
    input_vector)
    output_vector = activation_function(output_vector)
    output_vector = np.dot(self.who,
    output_vector)
    output_vector = activation_function(output_vector)
    return output_vector
  def evaluate(self, data, labels):
    corrects, wrongs = 0, 0
    for i in range(len(data)):
      res = self.run(data[i])
      res_max = res.argmax()
      if res_max == labels[i]:
        corrects += 1
      else:
        wrongs += 1
    return corrects, wrongs


In [ ]:
epochs = 10

ANN = NeuralNetwork(no_of_in_nodes = image_pixels,no_of_out_nodes = 10,
no_of_hidden_nodes = 100,
learning_rate = 0.15)
weights = ANN.train(train_imgs,
train_labels_one_hot,
epochs=epochs,
intermediate_results=True)
cm = ANN.confusion_matrix(train_imgs, train_labels)
print(ANN.run(train_imgs[i]))


In [ ]:
cm = list(cm.items())
print(sorted(cm))


In [ ]:
for i in range(epochs):
  print("epoch: ", i)
  ANN.wih = weights[i][0]
  ANN.who = weights[i][1]
  corrects, wrongs = ANN.evaluate(train_imgs, train_labels)
  print("accuracy train: ", corrects / ( corrects + wrongs))
  corrects, wrongs = ANN.evaluate(test_imgs, test_labels)
  print("accuracy: test", corrects / ( corrects + wrongs))

Version biaisé

In [ ]:
import numpy as np
@np.vectorize
def sigmoid(x):
  return 1 / (1 + np.e ** -x)
  activation_function = sigmoid
from scipy.stats import truncnorm
def truncated_normal(mean=0, sd=1, low=0, upp=10):
  return truncnorm((low - mean) / sd,
  (upp - mean) / sd,
  loc=mean,
  scale=sd)
class NeuralNetwork:
  def __init__(self,
    no_of_in_nodes,
    no_of_out_nodes,
    no_of_hidden_nodes,
    learning_rate,
    bias=None
    ):
    self.no_of_in_nodes = no_of_in_nodes
    self.no_of_out_nodes = no_of_out_nodes
    self.no_of_hidden_nodes = no_of_hidden_nodes
    self.learning_rate = learning_rate
    self.bias = bias
    self.create_weight_matrices()
  def create_weight_matrices(self):
    #"""
    #A method to initialize the weight
    #matrices of the neural network with
    #WITH BIAS NODES 209optional bias nodes"""
    bias_node = 1 if self.bias else 0
    rad = 1 / np.sqrt(self.no_of_in_nodes + bias_node)
    X = truncated_normal(mean=0,
    sd=1,
    low=-rad,
    upp=rad)
    self.wih = X.rvs((self.no_of_hidden_nodes,
    self.no_of_in_nodes + bias_node))
    rad = 1 / np.sqrt(self.no_of_hidden_nodes + bias_node)
    X = truncated_normal(mean=0, sd=1, low=-rad, upp=rad)
    self.who = X.rvs((self.no_of_out_nodes,
    self.no_of_hidden_nodes + bias_node))
  def train(self, input_vector, target_vector):
    #"""
    #input_vector and target_vector can
    #be tuple, list or ndarray
    #"""
    bias_node = 1 if self.bias else 0
    if self.bias:
    # adding bias node to the end of the inpuy_vector
      input_vector = np.concatenate((input_vector,
      [self.bias]) )
    input_vector = np.array(input_vector, ndmin=2).T
    target_vector = np.array(target_vector, ndmin=2).T
    output_vector1 = np.dot(self.wih,
    input_vector)
    output_hidden = activation_function(output_vector1)
    if self.bias:
      output_hidden = np.concatenate((output_hidden,
      [[self.bias]]) )
    output_vector2 = np.dot(self.who,
    output_hidden)
    output_network = activation_function(output_vector2)
    output_errors = target_vector - output_network
    # update the weights:
    tmp = output_errors * output_network * (1.0 - output_netwo
    rk)
    tmp = self.learning_rate * np.dot(tmp, output_hidden.T)
    self.who += tmp
    # calculate hidden errors:
    hidden_errors = np.dot(self.who.T,
    output_errors)
    # update the weights:
    tmp = hidden_errors * output_hidden * (1.0 - output_hidde
    n)
    if self.bias:
      x = np.dot(tmp, input_vector.T)[:-1,:]
    else:
      x = np.dot(tmp, input_vector.T)
    self.wih += self.learning_rate * x
  def run(self, input_vector):
    #"""
    #input_vector can be tuple, list or ndarray
    #"""
    if self.bias:
      # adding bias node to the end of the inpuy_vector
      input_vector = np.concatenate((input_vector, [1]) )
    input_vector = np.array(input_vector, ndmin=2).T
    output_vector = np.dot(self.wih,
    input_vector)
    output_vector = activation_function(output_vector)
    if self.bias:
      output_vector = np.concatenate( (output_vector,
      [[1]]) )
    output_vector = np.dot(self.who,
    output_vector)
    output_vector = activation_function(output_vector)
    return output_vector
  def evaluate(self, data, labels):
    corrects, wrongs = 0, 0
    for i in range(len(data)):
      res = self.run(data[i])
      res_max = res.argmax()
      if res_max == labels[i]:
        corrects += 1
      else:
        wrongs += 1
    return corrects, wrongs
ANN = NeuralNetwork(no_of_in_nodes=image_pixels,
no_of_out_nodes=10,
no_of_hidden_nodes=200,
learning_rate=0.1,
bias=None)
for i in range(len(train_imgs)):
  ANN.train(train_imgs[i], train_labels_one_hot[i])
for i in range(20):
  res = ANN.run(test_imgs[i])
  print(test_labels[i], np.argmax(res), np.max(res))


In [ ]:
corrects, wrongs = ANN.evaluate(train_imgs, train_labels)
print("accuracy train: ", corrects / ( corrects + wrongs))
corrects, wrongs = ANN.evaluate(test_imgs, test_labels)
print("accuracy: test", corrects / ( corrects + wrongs))


Version biaisé + epoch

In [ ]:
import numpy as np
@np.vectorize
def sigmoid(x):
return 1 / (1 + np.e ** -x)
activation_function = sigmoid
from scipy.stats import truncnorm
def truncated_normal(mean=0, sd=1, low=0, upp=10):
  return truncnorm((low - mean) / sd,
  (upp - mean) / sd,
  loc=mean,
  scale=sd)
class NeuralNetwork:
  def __init__(self,
    no_of_in_nodes,
    no_of_out_nodes,
    no_of_hidden_nodes,
    learning_rate,
    bias=None
    ):
    self.no_of_in_nodes = no_of_in_nodes
    self.no_of_out_nodes = no_of_out_nodes
    self.no_of_hidden_nodes = no_of_hidden_nodes
    self.learning_rate = learning_rate
    self.bias = bias
    self.create_weight_matrices()
  def create_weight_matrices(self):
    #"""
    #A method to initialize the weight matrices
    #of the neural network with optional
    #bias nodes"""
    bias_node = 1 if self.bias else 0
    rad = 1 / np.sqrt(self.no_of_in_nodes + bias_node)
    X = truncated_normal(mean=0, sd=1, low=-rad, upp=rad)
    self.wih = X.rvs((self.no_of_hidden_nodes,
    self.no_of_in_nodes + bias_node))
    rad = 1 / np.sqrt(self.no_of_hidden_nodes + bias_node)
    X = truncated_normal(mean=0,
    sd=1,
    low=-rad,
    upp=rad)
    self.who = X.rvs((self.no_of_out_nodes,
    WITH BIAS NODES 214
    self.no_of_hidden_nodes + bias_node))
  def train_single(self, input_vector, target_vector):
    #"""
    #input_vector and target_vector can be tuple,
    #list or ndarray
    #"""
    bias_node = 1 if self.bias else 0
    if self.bias:
    # adding bias node to the end of the inpuy_vector
    input_vector = np.concatenate( (input_vector,
    [self.bias]) )
    output_vectors = []
    input_vector = np.array(input_vector, ndmin=2).T
    target_vector = np.array(target_vector, ndmin=2).T
    output_vector1 = np.dot(self.wih,
    input_vector)
    output_hidden = activation_function(output_vector1)
    if self.bias:
    output_hidden = np.concatenate((output_hidden,
    [[self.bias]]) )
    output_vector2 = np.dot(self.who,
    output_hidden)
    output_network = activation_function(output_vector2)
    output_errors = target_vector - output_network
    # update the weights:
    tmp = output_errors * output_network * (1.0 - output_network)
    tmp = self.learning_rate * np.dot(tmp,
    output_hidden.T)
    self.who += tmp
    # calculate hidden errors:
    hidden_errors = np.dot(self.who.T,
    output_errors)

    # update the weights:
    tmp = hidden_errors * output_hidden * (1.0 - output_hidden)
    if self.bias:
      x = np.dot(tmp, input_vector.T)[:-1,:]
    else:
      x = np.dot(tmp, input_vector.T)
    self.wih += self.learning_rate * x
  def train(self, data_array,
    labels_one_hot_array,
    epochs=1,
    intermediate_results=False):
    intermediate_weights = []
    for epoch in range(epochs):
      for i in range(len(data_array)):
        self.train_single(data_array[i],
        labels_one_hot_array[i])
        if intermediate_results:
          intermediate_weights.append((self.wih.copy(),
          self.who.copy()))
    return intermediate_weights
  def run(self, input_vector):
    # input_vector can be tuple, list or ndarray
    if self.bias:
    # adding bias node to the end of the inpuy_vector
      input_vector = np.concatenate( (input_vector,
      [self.bias]) )
    input_vector = np.array(input_vector, ndmin=2).T
    output_vector = np.dot(self.wih,
    input_vector)
    output_vector = activation_function(output_vector)
    if self.bias:
      output_vector = np.concatenate( (output_vector,
      [[self.bias]]) )

    output_vector = np.dot(self.who,
    output_vector)
    output_vector = activation_function(output_vector)
    return output_vector
  def evaluate(self, data, labels):
    corrects, wrongs = 0, 0
    for i in range(len(data)):
      res = self.run(data[i])
      res_max = res.argmax()
      if res_max == labels[i]:
        corrects += 1
      else:
        wrongs += 1
    return corrects, wrongs
  


In [ ]:
epochs = 12
network = NeuralNetwork(no_of_in_nodes=image_pixels,
no_of_out_nodes=10,
no_of_hidden_nodes=100,
learning_rate=0.1,
bias=None)
weights = network.train(train_imgs,
train_labels_one_hot,
epochs=epochs,
intermediate_results=True)
for epoch in range(epochs):
  print("epoch: ", epoch)
  network.wih = weights[epoch][0]
  network.who = weights[epoch][1]
  corrects, wrongs = network.evaluate(train_imgs,
  train_labels)
  print("accuracy train: ", corrects / ( corrects + wrong
  s))
  corrects, wrongs = network.evaluate(test_imgs,
  test_labels)
  print("accuracy test: ", corrects / ( corrects + wrongs))

In [ ]:
epochs = 12
with open("nist_tests.csv", "w") as fh_out:
  for hidden_nodes in [20, 50, 100, 120, 150]:
    for learning_rate in [0.01, 0.05, 0.1, 0.2]:
      for bias in [None, 0.5]:
      network = NeuralNetwork(no_of_in_nodes=image_pixels,
      no_of_out_nodes=10,
      no_of_hidden_nodes=hidden_nodes,
      learning_rate=learning_rate,
      bias=bias)
      weights = network.train(train_imgs,
      train_labels_one_hot,
      epochs=epochs,
      intermediate_results=True)
        for epoch in range(epochs):
        print("*", end="")
        network.wih = weights[epoch][0]
        network.who = weights[epoch][1]
        train_corrects, train_wrongs = network.evaluat
        e(train_imgs,
        train_labels)
        test_corrects, test_wrongs = network.evaluat
        e(test_imgs,
        test_labels)
        outstr = str(hidden_nodes) + " " + str(learnin
        g_rate) + " " + str(bias)
        outstr += " " + str(epoch) + " "
        outstr += str(train_corrects / (train_correct
        s + train_wrongs)) + " "
        outstr += str(train_wrongs / (train_corrects
        + train_wrongs)) + " "
        outstr += str(test_corrects / (test_corrects
        + test_wrongs)) + " "
        outstr += str(test_wrongs / (test_corrects + t
        est_wrongs))
        fh_out.write(outstr + "\n" )
        fh_out.flush()


Réseau neuronal avec plusieurs couches cachées

In [ ]:
import numpy as np
from scipy.special import expit as activation_function
from scipy.stats import truncnorm
def truncated_normal(mean=0, sd=1, low=0, upp=10):
  return truncnorm((low - mean) / sd,
  (upp - mean) / sd,
  loc=mean,
  scale=sd)


In [ ]:
class NeuralNetwork:
  def __init__(self,
    network_structure, # ie. [input_nodes, hidden1_no
    des, ... , hidden_n_nodes, output_nodes]
    learning_rate,
    bias=None
    ):
    self.structure = network_structure
    self.learning_rate = learning_rate
    self.bias = bias
    self.create_weight_matrices()
  def create_weight_matrices(self):
    bias_node = 1 if self.bias else 0
    self.weights_matrices = []
    layer_index = 1
    no_of_layers = len(self.structure)
    while layer_index < no_of_layers:
      nodes_in = self.structure[layer_index-1]
      nodes_out = self.structure[layer_index]
      n = (nodes_in + bias_node) * nodes_out
      rad = 1 / np.sqrt(nodes_in)
      X = truncated_normal(mean=2,
      sd=1,
      low=-rad,
      upp=rad)
      wm = X.rvs(n).reshape((nodes_out, nodes_in + bias_node))
      self.weights_matrices.append(wm)
      layer_index += 1
  def train(self, input_vector, target_vector):
    #"""
    #input_vector and target_vector can be tuple,
    #list or ndarray
    #"""
    no_of_layers = len(self.structure)
    input_vector = np.array(input_vector, ndmin=2).T
    layer_index = 0
    # The output/input vectors of the various layers:
    res_vectors = [input_vector]
    while layer_index < no_of_layers - 1:
      in_vector = res_vectors[-1]
      if self.bias:
        # adding bias node to the end of the 'input'_vecto
        r
        in_vector = np.concatenate( (in_vector,
        [[self.bias]]) )
        res_vectors[-1] = in_vector
      x = np.dot(self.weights_matrices[layer_index],
      in_vector)
      out_vector = activation_function(x)
      # the output of one layer is the input of the next one:
      res_vectors.append(out_vector)
      layer_index += 1
    layer_index = no_of_layers - 1
    target_vector = np.array(target_vector, ndmin=2).T
    # The input vectors to the various layers

    output_errors = target_vector - out_vector
    while layer_index > 0:
      out_vector = res_vectors[layer_index]
      in_vector = res_vectors[layer_index-1]
      if self.bias and not layer_index==(no_of_layers-1):
        out_vector = out_vector[:-1,:].copy()
      tmp = output_errors * out_vector * (1.0 - out_vecto
      r)
      tmp = np.dot(tmp, in_vector.T)
      #if self.bias:
      # tmp = tmp[:-1,:]
      self.weights_matrices[layer_index-1] += self.learnin
      g_rate * tmp
      output_errors = np.dot(self.weights_matrices[layer_ind
      ex-1].T,
      output_errors)
      if self.bias:
        output_errors = output_errors[:-1,:]
      layer_index -= 1
  def run(self, input_vector):
    # input_vector can be tuple, list or ndarray
    no_of_layers = len(self.structure)
    if self.bias:
      # adding bias node to the end of the inpuy_vector
      input_vector = np.concatenate( (input_vector,
      [self.bias]) )
    in_vector = np.array(input_vector, ndmin=2).T
    layer_index = 1
    # The input vectors to the various layers
    while layer_index < no_of_layers:
      x = np.dot(self.weights_matrices[layer_index-1],
      in_vector)
      out_vector = activation_function(x)
      # input vector for next layer
      in_vector = out_vector
      if self.bias:
      in_vector = np.concatenate( (in_vector,
      [[self.bias]])
      )
      layer_index += 1
    return out_vector

  def evaluate(self, data, labels):
    corrects, wrongs = 0, 0
    for i in range(len(data)):
      res = self.run(data[i])
      res_max = res.argmax()
      if res_max == labels[i]:
        corrects += 1
      else:
        wrongs += 1
    return corrects, wrongs




In [ ]:
ANN = NeuralNetwork(network_structure=[image_pixels, 50, 50, 10],
learning_rate=0.1,
bias=None)
for i in range(len(train_imgs)):
  ANN.train(train_imgs[i], train_labels_one_hot[i])


In [ ]:
corrects, wrongs = ANN.evaluate(train_imgs, train_labels)
print("accuracy train: ", corrects / ( corrects + wrongs))
corrects, wrongs = ANN.evaluate(test_imgs, test_labels)
print("accuracy: test", corrects / ( corrects + wrongs))
